# Filtrar detecciones 

In [43]:
import os
import pandas as pd
import matplotlib.pyplot as plt

# Leer todos las detecciones para luego fusionarlas
folder_path = "./Detecciones_estandarizadas"
csv_files = [f for f in os.listdir(folder_path) if f.endswith('.csv')]
dfs = [pd.read_csv(os.path.join(folder_path, f)) for f in csv_files]
df_concat = pd.concat(dfs, ignore_index=True)

# Número de especies y proyectos
print(f"Número de especies: {df_concat['speciesName'].nunique()}")
print(f"Número de proyectos: {df_concat['projectName'].nunique()}")


Número de especies: 318
Número de proyectos: 2


## Filtrar especies objetivo

In [44]:
targets_df = pd.read_csv('/Users/santiagoruiz/Documents/Escuchaton/especies_objetivo.csv')
df_concat = df_concat[df_concat['speciesName'].isin(targets_df['Especies'])]

# Numero de clips por especie
df_concat['speciesName'].value_counts()

speciesName
Pulsatrix perspicillata       520
Zebrilus undulatus            361
Ciccaba virgata               321
Lophostrix cristata           210
Ceratopipra erythrocephala    162
Name: count, dtype: int64

## Filtar por puntaje de deteccion (confidence)

En este paso se van a filtrar un numero especifico de detecciones para cada decil de confidece. Es decir, todos los valores de confianza por especie van a agruparse en 10 partes iguales y luego se van a seleccionar de forma aleatoria un numero determinado de detecciones por decil (clips_por_especie_decil)

In [45]:
clips_por_especie_decil = 10

for species, group in df_concat.groupby('speciesName'):
    group = group.copy()
    group['conf_bin'] = pd.qcut(group['confidence'], 10, labels=False, duplicates='drop')
    sampled = group.groupby('conf_bin').apply(lambda x: x.sample(n=min(clips_por_especie_decil, len(x)), random_state=42)).reset_index(drop=True)

    output_folder = "./Detecciones_filtradas"
    species_folder = os.path.join(output_folder)
    os.makedirs(species_folder, exist_ok=True)
    del sampled['conf_bin']
    output_path = os.path.join(species_folder, f"{species}.csv")
    sampled.to_csv(output_path, index=False)

/var/folders/_6/xzrhlh5s5yd6q142xlf57x4h0000gn/T/ipykernel_75026/3275535847.py:6: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  sampled = group.groupby('conf_bin').apply(lambda x: x.sample(n=min(clips_por_especie_decil, len(x)), random_state=42)).reset_index(drop=True)
/var/folders/_6/xzrhlh5s5yd6q142xlf57x4h0000gn/T/ipykernel_75026/3275535847.py:6: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  sampled = gr